In [0]:
from rpy2.robjects import r
import rpy2.robjects as robjects
import sys
from datetime import datetime

In [3]:
share_holders_count = input("주주의 수를 입력하세요: ")
accounts_receivable = input("매출채권의 총액을 입력하세요: ")
established_date_string = input("설립일자를 입력하세요(ex)20200202: ")

주주의 수를 입력하세요: 1
매출채권의 총액을 입력하세요: 2500000
설립일자를 입력하세요(ex)20200202: 19980212


In [7]:
r("install.packages(\'KMsurv\')")
r("library(KMsurv)")
r("library(survival)")
r("library(ggplot2)")
r("install.packages(\'survminer\')")
r("library(survminer)")
r("install.packages(\'car\')")
r("library(car)")
r("install.packages(\'maxstat\')")
r("library(maxstat)")
r("install.packages(\'party\')")
r("library(party)")

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://cran.rstudio.com/src/contrib/KMsurv_0.1-5.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Content type 'application/x-gzip'
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 70896 bytes (69 KB)

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: =
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist

'party','struccha...,'sandwich','zoo',...,'utils','datasets','methods','base'


In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [16]:
#failure 열은 부도가 나면 1, 부도가 나지 않으면 0 을 의미
r("df = read.csv('/content/drive/My Drive/result_survival.csv',header=T)")
r("df[,\"failure\"] = ifelse(is.na(df$suspension_date),0,1)")

0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# censoring 의 경우를 위해 suspension date 가 없는 경우 현재 날짜(2020.02.05) 를 입력한다
r("df$suspension_date = ifelse(is.na(df$suspension_date),\"20200205\",df$suspension_date)")

'20200205','20200205','20200205','20200205',...,'20200205','20200205','20191231','20200205'


In [18]:
r("df$establish_date=as.character(df$establish_date)")

'20111104','20140520','19861212','20051109',...,'20110504','20110125','19920709','20040712'


In [19]:
#time 열은 기업 설립일로부터 부도날짜/현재날짜를 의미한다
r("df[,\"time\"] = as.integer(as.Date(df$suspension_date,format=\"%Y%m%d\")-as.Date(df$establish_date,format = \"%Y%m%d\"))")

"3,015","2,087","12,108","5,201",...,"3,199","3,298","10,036","5,686"


In [0]:
# survival function prediction

In [20]:
# 생존분석 결과 남은 변수로 부도율을 예측하자
r("data2 <- df[,c(\"ked_code\",\"business_number\",\"accounts_receivable\",\"share_holders_count\",\"failure\",\"time\")]")
r("data2 <- na.omit(data2)")

ked_code,business_number,accounts_receivable,share_holders_count,failure,time
211080.000000,1130539975.000000,"568,936",0,0.000000,"7,474"
258477.000000,1250689233.000000,"569,000",0,0.000000,"7,928"
602757.000000,6210847400.000000,"1,768,384",0,0.000000,"6,485"
504204.000000,5040290162.000000,"260,314",0,0.000000,"16,532"
...,...,...,...,...,...
7451428.000000,1248713501.000000,"953,000",0,0.000000,"3,199"
7188951.000000,1138645592.000000,"1,618,812",0,0.000000,"3,298"
126406.000000,5058108511.000000,"304,182",0,1.000000,"10,036"
1086840.000000,4088162742.000000,"50,723",0,0.000000,"5,686"


In [21]:
# share_holders_count 와 accounts_receivable 변수로 coxph 를 적용한다
r("test<-coxph(Surv(time,failure)~share_holders_count+accounts_receivable,data=data2)")

R object with classes: ('coxph',) mapped to:
<ListVector - Python:0x7f167b967988 / R:0x7542f20>
[FloatVector, Matrix, FloatVector, FloatVector, ..., Matrix, BoolVector, Formula, Vector]
  coefficients: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b979e48 / R:0x10749bc8>
[-0.355813, -0.000000]
  var: <class 'rpy2.robjects.vectors.Matrix'>
  R object with classes: ('matrix',) mapped to:
<Matrix - Python:0x7f167b968948 / R:0xb5aba98>
[0.014393, 0.000000, 0.000000, 0.000000]
  loglik: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b98b1c8 / R:0x107498c8>
[-1548.830982, -1537.056906]
  score: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b973048 / R:0x103768c8>
[15.588995]
...
  linear.predictors: <class 'rpy2.robjects.vectors.Matrix'>
  R object with classes: ('Surv',) mapped to:
<Matrix - Python:0x7f167b973648 / R:0xd6c6510>
[7474.000000, 7928.000000, 6485.000000, 16532.000000, ..., 0.000000, 0.000000, 1.000000, 0.000000]
  residuals: <class 'rpy2.robjects.vectors.BoolVector'>
  R object with classes: ('logical',) mapped to:
<BoolVector - Python:0x7f166ce55b08 / R:0x116331c0>
[       1]
  means: <class 'rpy2.robjects.Formula'>
  R object with classes: ('formula',) mapped to:
<Formula - Python:0x7f166ce559c8 / R:0xcf41620>
  method: <class 'rpy2.robjects.vectors.Vector'>
  R object with classes: ('coxph',) mapped to:
<Vector - Python:0x7f166ce55d48 / R:0xd84f8c0>
[RObject, Vector, RObject]

In [22]:
# 특정 값을 가진 경우 생존함수를 그려본다
r("predict<-survfit(test,c(share_holders_count=" + str(share_holders_count) + ",accounts_receivable=" + str(accounts_receivable) + "))")

R object with classes: ('survfitcox', 'survfit') mapped to:
<ListVector - Python:0x7f166ce55408 / R:0x11f4afd8>
[IntVector, FloatVector, FloatVector, FloatVector, ..., FloatVector, StrVector, FloatVector, Vector]
  n: <class 'rpy2.robjects.vectors.IntVector'>
  R object with classes: ('integer',) mapped to:
<IntVector - Python:0x7f167b9c6f48 / R:0xa7d17a8>
[614]
  time: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b979348 / R:0xbe333f0>
[369.000000, 544.000000, 586.000000, 744.000000, ..., 20112.000000, 23622.000000, 24997.000000, 33486.000000]
  n.risk: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b979308 / R:0x100e6800>
[614.000000, 613.000000, 612.000000, 611.000000, ..., 4.000000, 3.000000, 2.000000, 1.000000]
  n.event: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b979148 / R:0x6332000>
[0.000000, 0.000000, 0.000000, 0.000000, ..., 0.000000, 0.000000, 0.000000, 0.000000]
...
  surv: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b967748 / R:0xfa2c800>
[1.000000, 1.000000, 1.000000, 1.000000, ..., 0.230565, 0.230565, 0.230565, 0.230565]
  cumhaz: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
<StrVector - Python:0x7f167b9684c8 / R:0xf6e4f20>
['log']
  std.err: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7f167b968f88 / R:0xf6ef498>
[0.950000]
  logse: <class 'rpy2.robjects.vectors.Vector'>
  R object with classes: ('survfitcox', 'survfit') mapped to:
<Vector - Python:0x7f167b968508 / R:0xc14ce08>
[RObject, RObject, Vector]

In [0]:
predict = robjects.r['predict']

In [24]:
predict.names[5]

'surv'

In [0]:
today = datetime.today()

In [0]:
established_date = datetime.strptime(established_date_string, '%Y%m%d')

In [0]:
operation_days = (today-established_date).days

In [0]:
# 각 관측시간을 모아놓은 벡터
times = predict[1]
survs = predict[5]

In [29]:
r('''
surv_prob <- function(dt,day) {
  Y = 0; N = 0
  for(i in 1:614){
    predict<-survfit(test,c(share_holders_count=dt$share_holders_count[i],
                            accounts_receivable=dt$accounts_receivable[i]))
    
    save = predict$surv[which(abs(predict$time-day) == min(abs(predict$time-day)))]
    
    if(dt$failure[i] == 1){
      Y = Y+save} else {
        N = N + save
        }
  }
  result = c(Y/293,N/321)
  names(result) = c("bankruptcy_Y_prob_mean","bankruptcy_N_prob_mean")
  return(result)
}
''')

R object with classes: ('function',) mapped to:
<SignatureTranslatedFunction - Python:0x7f166ce46608 / R:0xbbcc318>

In [30]:
r("comparables <- surv_prob(data2, "+str(operation_days)+")")

0.537207,0.562305


In [0]:
comparables = robjects.r['comparables']

In [32]:
min_diff = sys.maxsize
min_index = -1
for i in range(len(times)):
    diff = abs(times[i]-operation_days)
    if(diff<min_diff):
        min_diff = diff
        min_index = i
if(min_index==-1):
    print("측정 불가")
else:
    surv_probability = survs[min_index]
    print("생존 확률: " + str(surv_probability))

생존 확률: 0.6412265535329057


In [33]:
print("같은 기간 동안 운영된 기업의 평균 생존확률")
print("부도가 난 경우: " + str(comparables[0]))
print("부도가 나지 않은 경우: " + str(comparables[1]))

같은 기간 동안 운영된 기업의 평균 생존확률
부도가 난 경우: 0.537207159070066
부도가 나지 않은 경우: 0.5623045790534018
